<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;a:link{color: white}">
    <h1 style='color:GhostWhite;'>Part 1: Should This Loan be Approved or Denied ?</h1>

An XGBoost v1.6+ data model to predict whether a loan can be approved or denied.

Optuna hyperparameter tuning for the XGBoost model is in Part 2 => <a style="color:yellow" href="https://www.kaggle.com/code/josephramon/sba-optuna-xgboost">Part 2</a><br>
    </div>

<div class="alert alert-block alert-success">  
    <b>Dataset Source</b><br><br>
    <a href="https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied">U.S. Small Business Administration (SBA) Dataset</a>
<br><br>
    All information about the dataset can be found at the <b>above link</b><br><br>    
    *<i>Thanks to Hamza for his <a href="https://www.kaggle.com/code/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna/notebook">Notebook on Optuna</a> which was used as a guide.</i> 
<br><br>
    If interested, Data Exploratory Visualization in Tableau can also be seen at :<br>
    <a href= "https://public.tableau.com/app/profile/joseph8038/viz/SBADatasetVisualizationandAnalysis/SBADatasetVisualizationandAnalysis-StoryBoard">SBA Data Exploratory Visualization in Tableau</a>
</div>

<div class="alert alert-block alert-info" style="color:DarkSlateBlue">
This notebook is divided into 2 main parts:<br>
<ul>
<li><a style="color:DarkSlateGrey;" href="#part1"><b>Part 1: Pipeline</b></a> - this is the end result encapsulated into a pipeline</li><br>
<li><a style="color:DarkSlateGrey" href="#part2"><b>Part 2: Data Exploration (EDA) and Preparation, Modeling, Metrics</b></a> - from start to end, with some notes</li><br>
</ul>
"Our model results are way more dependent on how well feature engineering is performed than on the model itself. Machine Learning models are like very skilled linguists that can decipher any text in any language. However, it will not be helpful if they are handed a bunch of scribbles or blurred out text. EDA should not be skipped, as a thorough EDA and feature engineering process accounts for 90% of the results of a good model."<br><br>
One method of avoiding memory leaks is doing processing inside a function. It creates a new scope for the intermediate variables and removes them automatically when the interpreter exits the function; hence, most of the code below are encapsulated into functions for this purpose. 
</div>

<h2>Table Of Contents</h2>
<ul>
    <li><a style="color:DarkSlateGrey" href="#paths_and_flags">Paths and Flags</a></li>
    <li><a style="color:DarkSlateGrey" href="#libraries">Libraries</a></li>   
    <li><a style="color:DarkSlateGrey" href="#functions">Custom Classes</a></li>
    <li><a style="color:DarkSlateGrey" href="#xgboost_class">XGBoost Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#other_models">Other Models Class</a></li>
    <br>
    <li><a style="color:DarkSlateGrey" href="#part1">Part 1. PipeLine</a></li>
    <ul>
        <li><a style="color:DarkSlateGrey" href="#pl_classes">Pipeline Classes</a></li>
        <li><a style="color:DarkSlateGrey" href="#load_pl_df">Load Dataset for PipeLine</a></li>
        <li><a style="color:DarkSlateGrey" href="#pl_run">Run the pipeline</a></li>
    </ul>
    <br>
    <li><a style="color:DarkSlateGrey" href="#part2">Part 2. Data Exploration and Preparation, Modeling, Metrics</a></li>
    <ul>
        <li><a style="color:DarkSlateGrey" href="#de_load_df">Load Dataset</a></li>
        <li><a style="color:DarkSlateGrey" href="#dep">Data Exploration / Preparation</a></li>
        <li><a style="color:DarkSlateGrey" href="#build_model">Build Model Using XGBoost</a></li>
        <ul>
            <li><a style="color:DarkSlateGrey" href="#model1">Model v1</a></li>
            <li><a style="color:DarkSlateGrey" href="#oversample">Oversample</a></li>
            <ul>
                <li><a style="color:DarkSlateGrey" href="#model2">Model v2</a></li>
                <li><a style="color:DarkSlateGrey" href="#model3">Model v3</a></li>
            </ul>
        </ul>
        <li><a style="color:DarkSlateGrey" href="#test_model">Test Model</a></li>
        <ul>
            <li><a style="color:DarkSlateGrey" href="#test_test_dataset">Test Model With Test Dataset</a></li>
           <li><a style="color:DarkSlateGrey" href="#test_user_input">Test Model With User Input</a></li>
        </ul>
        <li><a style="color:DarkSlateGrey" href="#mutual_info">Mutual Information Scores</a></li>
        <li><a style="color:DarkSlateGrey" href="#trim_datasets">Trim Datasets</a></li>
        <li><a style="color:DarkSlateGrey" href="#results1">Full or Trimmed Dataset</a></li>
    </ul>  
    <br>
    <li><a style="color:DarkSlateGrey" href="#part2" target="_blank">Optuna Hyperparameter Tuning - in Part 2 of this notebook</a></li>
</ul>

<a id="paths_and_flags"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Paths and Flags</b></div>

In [1]:
import os
'''
kaggle_flag :
   0 - if running outside Kaggle (e.g. Jupyter Notebook), change filepath & savepath to your 
       own path
   1 - if running as a Kaggle notebook
'''
# Change this logic to your own if needed
if os.path.exists('../usr/lib/myfuncs/myfuncs.py'):
    kaggle_flag = 1
    print('Running a Kaggle notebook')
else:
    kaggle_flag = 0
    print('Not running a Kaggle notebook')
    
# alert_flag - change to 0 for no sound alert, 1 for sound alert after long running cells
alert_flag = 0

# GPU is automatically detected if activated

#---------------------------------------------------------------------------------------#

if kaggle_flag == 1:             # Kaggle
    filepath  = "../input/should-this-loan-be-approved-or-denied/"
    savepath  = "./"
    final_ds  = '../input/sba-xgboost-model/sba_final.csv.feather'  # imported from Part 1 Notebook
    final_csv = '../input/sba-xgboost-model/sba_final.csv'          # imported from Part 1 Notebook
    functions_path = "../usr/lib/myfuncs/myfuncs.py"
else:
    filepath  = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
    savepath  = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
    final_ds  = f'{savepath}sba_final.csv.feather'
    final_csv = f'{savepath}sba_final.csv'
    functions_path = 'C:\\Python\\Python_Data_Science_Exercises\\mylibs\\'

audio_path="https://www.soundjay.com/misc/sounds/tablet-bottle-1.mp3" # for alert

Not running a Kaggle notebook


<a id="libraries"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Libraries</b></div>

In [2]:
from IPython.display import clear_output   # to be able to use clear_output(wait=True)
def install_packages():
    print('Please wait, package installations started, if needed')
    libs = ['scikit-learn', 'seaborn', 'numpy','matplotlib', 'tensorflow','torch','joblib',
            'psutil','imbalanced-learn','xgboost','optuna','pyarrow','pyttsx3',
            'pandas-profiling','sweetviz','dataprep','pympler','memory_profiler','line_profiler',
            'multiprocessing']
    
    piplist = !pip list
    for i in range(len(libs)):
        if not piplist.grep(libs[i]):
            !pip3 install {libs[i]}

    clear_output(wait=True)
    print('Package installations completed')

install_packages()

Package installations completed


In [3]:
import pandas as pd
#import modin.pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pyttsx3
from IPython.display import Audio, display
from IPython.display import FileLink
from IPython.display import IFrame
from IPython.core.display import HTML
import hashlib
import copy                     # for deepcopy()
import datetime as dt
import optuna
import gc
from pandas_profiling import ProfileReport
import sweetviz as sv
import shutil
import psutil
import os
import sys
import pickle
import joblib
from pprint import pprint
import torch             # for clearing GPU cache
from time import sleep
import multiprocessing as mp
from pympler import muppy       # for memory profiling
from pympler import summary     # for memory profiling
from pympler.classtracker import ClassTracker
import xgboost
%load_ext line_profiler
%load_ext memory_profiler
%matplotlib inline
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

clear_output(wait=True)
print('Package imports completed')

Package imports completed


In [4]:
# Kernel must be restarted if XGBoost is upgraded
# importlib.reload and %autoreload do not work, so manually restart 
# This check is basically for Kaggle which has an older version of XGBoost, as at April 2022
if '1.6' not in xgboost.__version__:
    !pip3 install --upgrade xgboost
    clear_output(wait=True)
    print('XGBoost Package upgrade completed.  KERNEL RESTART NEEDED FOR NOTEBOOK.')
else:
    print('XGBoost version is already 1.6+')

XGBoost version is already 1.6+


In [5]:
# XGBoost version should be 1.6+ and up
assert '1.6' in xgboost.__version__,\
    "XGBoost version must be 1.6+. RESTART KERNEL if already upgraded."

print(f'XGBoost __Version__ : {xgboost.__version__}')
print()
!pip3 show xgboost

XGBoost __Version__ : 1.6.0

Name: xgboost
Version: 1.6.0
Summary: XGBoost Python Package
Home-page: https://github.com/dmlc/xgboost
Author: 
Author-email: 
License: Apache-2.0
Location: c:\programdata\anaconda3\lib\site-packages
Requires: scipy, numpy
Required-by: autoxgb


In [6]:
# ensure garbage collector is enabled
(gc.isenabled() == False) and gc.enable();

<a id="functions"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Custom Functions and Classes</b></div>

In [7]:
sys.path

['C:\\Python\\Python_Data_Science_Exercises',
 'C:\\ProgramData\\Anaconda3\\python38.zip',
 'C:\\ProgramData\\Anaconda3\\DLLs',
 'C:\\ProgramData\\Anaconda3\\lib',
 'C:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\latri\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin']

In [8]:
# import custom functions
# RESTART kernel if myfuncs is modified
if functions_path not in sys.path:
    sys.path.append(functions_path)

from myfuncs import *

print('Custom functions import completed')

Custom functions import completed


<div class="alert alert-block alert-info">
<b>Custom functions and classes in <a style="color:ForestGreen" href="https://www.kaggle.com/code/josephramon/myfuncs" target="_blank">myfuncs.py</a></b>.<br>  
In Kaggle, myfuncs.py is set up as a <b>Utility Script</b> in /usr/lib<br>
<ul>
    <li>is_kaggle_gpu_enabled()</li>
<li>clear_gpu(tree_method='gpu_hist')</li>
<li>reduce_mem_usage(df, print_info = True, use_float16=False)</li>
<li>runtime(rt1,rt2)</li>
<li>create_download_link(title = "Download ", filename = "data.csv")</li>
<li>GetRam()</li>
<li>convertFloatToDecimal(f=0.0, precision=2)</li>
<li>formatFileSize(size, sizeIn, sizeOut, precision=0)</li>
<li>check_cols_with_nulls(df)</li>
<li>check_infinity_nan(df, dfname)</li>
<li>fixvals(val)</li>
<li>model_eval(y_valid,predictions, cmDisplay='False')</li>
<li>plot_features(booster, figsize)</li>
<li>make_mi_scores(X, y)</li>
<li>plot_mi_scores(scores)</li>
<li>GetSweetVizReport(df, htmlpath, kaggle_flag)</li>
<li>SetVoice(kaggle_flag)</li>
<li>class color
</div>

In [9]:
gpu_enabled = is_kaggle_gpu_enabled()

if gpu_enabled == False:
    tree_method = 'hist'
else:
    tree_method = 'gpu_hist'

del gpu_enabled
gc.collect()

sleep(5)
clear_output(wait=True)
tree_method

'hist'

In [ ]:
''' 
Set up voice object.  Used in different areas of notebook to indicate completion of long processes.
'''
engine = SetVoice(kaggle_flag)

<a id="xgboost_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>XGBoost Class</b></div>

In [10]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split
#from xgboost import XGBRegressor
from xgboost import XGBClassifier

class process_model():  
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.y_train = None, None
        self.X_valid, self.X_test = None, None
        self.y_valid, self.y_test = None, None

        print(f'MIS_Status Count ->  1 : {Counter(y)[1]}, 0 : {Counter(y)[0]}')
    
    # oversampling method
    def osample(self, print_info = True):
        # define oversampling strategy
        oversample = RandomOverSampler(sampling_strategy='minority') 
        if print_info == True:
            print('X size : ', len(self.X))
            print('y size : ', len(self.y))
        # fit and apply the transform
        X_over, y_over = oversample.fit_resample(self.X, self.y)

        # summarize class distribution
        if print_info == True:
            print(f'Before Oversampling -> 1 : {Counter(self.y)[1]}, 0 : {Counter(self.y)[0]}')
            print(f'After Oversampling  -> 1 : {Counter(y_over)[1]}, 0 : {Counter(y_over)[0]}')
        
        # update X and y with the oversampled results 
        self.X = X_over
        self.y = y_over
        
        # return the oversampled results in case they are needed in another module
        #return {'X_over':X_over, 'y_over':y_over}
    
    def split_data(self, X_size = 0.7):   
        # Split Data into Train:Validate:Test
        
        # train_size=X_size
        # In the first step, we will split the data in training and remaining dataset
        self.X_train, X_rem, self.y_train, y_rem = train_test_split(self.X, self.y,
                                                        train_size = X_size, random_state=48) 

        # Now since we want the valid and test size to be equal,
        # we have to define valid_size=0.5 (that is 50% of remaining data)
        # test_size = 0.5

        self.X_valid, self.X_test, self.y_valid, self.y_test = train_test_split(X_rem,y_rem,
                                                            test_size=0.5, random_state=48)
        
        return {'X_train':self.X_train, 'y_train':self.y_train,
                'X_valid':self.X_valid, 'y_valid':self.y_valid,
                'X_test':self.X_test, 'y_test':self.y_test}
    
    # Method to run model 
    # desc - description of metrics report
    def prep_run_model(self, desc='Metrics', cmDisplay=False, PipeLine_flag = False,
                hyperparams = {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 6,
                               'tree_method':tree_method, 'early_stopping_rounds':100,
                               'eval_metric':['logloss','error']}):
        # from XGBoost 1.6, early_stopping_rounds and eval_metric are under parameters,
        # and deprecated from fit() method.
        # The default hyperparameters are conservative, to help avoid overfitting
        
        print()
        print(f"{color.bold}Please wait, Fitting model can take time ...{color.end}")
        
        '''
        XGBRegressor is for continuous target/outcome variables. These are often called 
        "regression problems."

        XGBClassifier is for categorical target/outcome variables. These are often called 
        "classification problems."
        
        xg_model = XGBRegressor(n_estimators = self.mn_estimators,
                                learning_rate = self.mlearning_rate,
                                max_depth = self.mmax_depth,
                                n_jobs=4)
        
        xg_model = XGBClassifier(n_estimators = self.mn_estimators,
                                learning_rate = self.mlearning_rate,
                                max_depth = self.mmax_depth,
                                use_label_encoder =False,
                                n_jobs=4)
        '''
        
        if PipeLine_flag == True:
            # hyperparams is a result of Optuna hyperparameter tuning (Part 3 of this notebook)
            # the hyperparameters lean towards being conservative to help avoid overfitting
            hyperparams = { 'tree_method': tree_method,
                            'lambda': 0.023437933789759252,
                            'alpha': 0.005813454622750776,
                            'gamma': 0,
                            'colsample_bytree': 0.9,
                            'subsample': 1.0,
                            'learning_rate': 0.05,
                            'n_estimators': 1000,
                            'max_depth': 13,
                            'random_state': 48,
                            'min_child_weight': 1,
                            'early_stopping_rounds': 100.0,
                            'eval_metric': 'error'}
            
        xg_model = XGBClassifier(**hyperparams,use_label_encoder =False)
       
        #eval_setparam = [(self.X_train, self.y_train), (self.X_valid, self.y_valid)]
        eval_setparam = [(self.X_valid, self.y_valid)]
        
        xg_model.fit(self.X_train, self.y_train,  
                     eval_set = eval_setparam,
                     verbose=False)
        
        gc.collect()
        clear_gpu()
 
        print("Fitting model completed.")
        print()
        print('Preparing Predictions')
    
        # Get predictions
        predictions = xg_model.predict(self.X_valid)
    
        print()
        print(f'{color.underline}{desc}{color.end}')

        eval_results = model_eval(self.y_valid, predictions, cmDisplay)
            
        # Return these values as they will be needed for further testing or metrics
        # in dictionary form to remember easier 
        return {'xg_model':xg_model,'predictions':predictions,
                    'X_train':self.X_train, 'y_train':self.y_train,
                    'X_valid':self.X_valid, 'y_valid':self.y_valid,
                    'X_test':self.X_test, 'y_test':self.y_test, 'eval_results':eval_results}

<a id="other_models"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Other Models Class</b></div>

In [11]:
from sklearn.ensemble import RandomForestClassifier

# inherit from XGBoost class (process_model)
class other_models(process_model):  
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.y_train = None, None
        self.X_valid, self.X_test = None, None
        self.y_valid, self.y_test = None, None

    #    print(f'MIS_Status Count ->  1 : {Counter(y)[1]}, 0 : {Counter(y)[0]}')
    
    # Method to run model 
    # desc - description of metrics report
    def prep_run_model(self, desc='Metrics', modelname = 'rfc',
                       hparams = {'n_estimators':100, 'random_state':48, 'max_depth':10},
                       cmDisplay=False):
        print()
        print(f"{color.bold}Please wait, Fitting model can take time ...{color.end}")  

        if modelname == 'rfc':
            model = RandomForestClassifier(**hparams) 
            model.fit(self.X_train, self.y_train)
            
        print("Fitting model completed.")
        print()
        print('Preparing Predictions')
    
        # Get predictions
        predictions = model.predict(self.X_valid)
    
        print()
        print(f'{color.underline}{desc}{color.end}')

        eval_results = model_eval(self.y_valid, predictions, cmDisplay)
        
        gc.collect()
        clear_gpu()
        
        # Return these values as they will be needed for further testing or metrics
        # in dictionary form to remember easier 
        return {'model':model,'predictions':predictions,
                    'X_train':self.X_train, 'y_train':self.y_train,
                    'X_valid':self.X_valid, 'y_valid':self.y_valid,
                    'X_test':self.X_test, 'y_test':self.y_test, 'eval_results':eval_results}

<a id="part1"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 id="part1" style='color:GhostWhite;'>Part 1. Pipeline</h1>
This pipeline handles both X and y
</div>

<a id="pl_classes"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>PipeLine Classes</b></div>

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
  
class PL_Object():
    def __init__(self,X,y):
        #store X and Y
        self.X=X
        self.y=y

class PreProcessor(BaseEstimator, TransformerMixin):
    def __init__(self,operation= 'X'):
        self.operation=operation
    @staticmethod
    def enabled(**kwargs):
        return True

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # check the parameters and return X and y inside the object
        X_data=X.X
        y_data=X.y
        
        print()
        print(f'{color.bdunl}PreProcessor initiated for {self.operation}{color.end}')
        
        #  do some work and assign it back to the X object which contains both X and y data
        if self.operation=='X':
            '''
            # NOTE: 'MIS_Status' is the target (y), but still in X, as we need to drop rows
                    with NaNs. We cannot do it separately, as there will be a mismatch in count 
                    of rows.  At the end of this procedure, we separate the new target data from X 
                    and update y.
            '''
            
            # Drop Na from rows
            #---------------------
            print(f'{color.bdblue}Drop Na{color.end}')
            X_data.dropna(subset=['DisbursementDate', 'NewExist', 'City', 'State',
                        'LowDoc', 'Name', 'NAICS', 'CreateJob', 'RetainedJob', 'FranchiseCode',
                        'UrbanRural', 'NoEmp', 'Term', 'MIS_Status'], how='any', inplace=True)
            
            # drop invalid classifications
            print('   Drop invalid classifications')
            X_data = X_data[(X_data['LowDoc'] == 'Y') | (X_data['LowDoc'] == 'N')]
            
            X_data = X_data[(X_data['NewExist'] == 1) | (X_data['NewExist'] == 2)]   
            
            # Trim leading and trailing spaces
            #---------------------------------
            print('   Trim leading and trailing spaces, if any')
            X_data['City'] = X_data['City'].str.strip()
            
            # Change dtype for columns needed for calculation or string extraction 
            #------------------------------------------------------------------------
            print(f'{color.bdgreen}Change dtype for columns needed for calculation or string extraction{color.end}')
            X_data = X_data.astype({'DisbursementGross':np.float64,'SBA_Appv':np.float64,
                              'GrAppv':np.float64, 'ChgOffPrinGr':np.float64,
                              'NAICS':np.str_, 'NewExist':np.int8})
            
            # Drop Duplicate Rows
            #------------------------------------------------------------------------
            print(f'{color.bdblue}Drop Duplicate Rows{color.end}')
            dupl_series = X_data.duplicated()
            num_of_dupl = len(X_data[dupl_series == True])
            if num_of_dupl > 0:
                X_data.drop_duplicates(inplace=True)
        
            # Create New Features
            #-----------------------
            print(f'{color.bdblue}Create New Features{color.end}')
            X_data['Industry'] = X_data['NAICS'].str[0:2]
            X_data = X_data.astype({'Industry':np.int8})
            
            X_data['Recession'] = np.where((X_data['DisbursementDate'] >= '2007-09-01')\
                     & (X_data['DisbursementDate'] <= '2009-06-30'), 1, 0)
            
            X_data['RealEstate'] = np.where(X_data['Term'] >= 240, 1, 0)
            
            X_data['SBA_Portion']=(X_data['SBA_Appv']/X_data['GrAppv']) * 100
            
            X_data["CityState"] = X_data["City"] + "_" + X_data["State"]
            
            print()
            print(f"X length = {len(X_data)}")
            print(f"Y length = {len(X_data['MIS_Status'])}")
            
            # Update X object
            X.X = X_data                      # type DataFrame
            X.y = X_data.pop('MIS_Status')    # type series
            
        elif self.operation=='y':
            pass                      
        else:
            pass
        
        #return modified X object
        return X
    

class EncodeCategorical(BaseEstimator, TransformerMixin):
    def __init__(self,operation= 'X'):
        self.operation=operation
    @staticmethod
    def enabled(**kwargs):
        return True

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # encode categorical features and return X and y inside the object
        X_data=X.X
        y_data=X.y
        
        print()
        print(f'{color.bdunl}Encode Categorical Features initiated for {self.operation}{color.end}')
        
        #  do some work and assign it back to the X object
        if self.operation=='X':         
            X_data['LowDoc'] = np.where((X_data['LowDoc'] == 'Y'), 1, 0)
            
            len_data=len(X_data)
            #cols_to_drop = []
            hash_constant = 900000   # fixed value so we can programmatically reproduce the hash
            #for col in X_data.columns:
            for col in X_data[['State','CityState']]:
                if X_data[col].dtype == 'object':
                    print(f'Column {col} has {X_data[col].nunique()} values among {len_data}')

                    if X_data[col].nunique() < 25:
                        print(f'One-hot encoding of {col}')
                        one_hot_cols = pd.get_dummies(X_data[col])
                        for ohc in one_hot_cols.columns:
                            X_data[col + '_' + ohc] = one_hot_cols[ohc]
                    else:
                      print(f'Hashing of {col}')
                      X_data[col + '_hash'] = X_data[col].apply(lambda row: int(hashlib.sha1((col +\
                                "_" + str(row)).encode('utf-8')).hexdigest(), 16) % hash_constant)

            X.X = X_data
            
        elif self.operation=='y':
            y_data = np.where(y_data == 'P I F', 1, 0)
            
            y_data = y_data.astype(np.int8)
            
            # convert back to series
            y_data = pd.Series(y_data)

            X.y = y_data                      
        else:
            pass
        #return modified X
        return X    

class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self,operation= 'X'):
        self.operation=operation
    @staticmethod
    def enabled(**kwargs):
        return True

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_data=X.X
        
        print()
        print(f'{color.bdunl}Drop Columns initiated for {self.operation}{color.end}')
        
        #  do some work and assign it back to the X object
 
        # Dropping 'City' as 'CityState_hash' is more ideal
        # Zip code has invalid values like 1, 2.  If we pad 0000 to 1, it's still not correct,
        # as state should be Alaska. Zip code 1 is different states in the dataset
        cols_to_drop = ['LoanNr_ChkDgt', 'Zip', 'Bank', 'BankState', 'ApprovalDate',
                        'ApprovalFY', 'ChgOffDate', 'BalanceGross', 'NAICS', 'ChgOffPrinGr',
                        'Name', 'RevLineCr', 'DisbursementDate', 'City', 'State', 'CityState',
                        'GrAppv']

        X_data.drop(columns=cols_to_drop, inplace=True)
            
        print()
        print('Unneeded Columns Dropped')
        
        # reduce mem usage of X_data as final step
        X_data = reduce_mem_usage(X_data)
        
        print()
        print(X_data.info())

        X.X = X_data
            
        #return modified X
        return X    

class XGBoost(BaseEstimator, TransformerMixin):
    def __init__(self,operation= 'X'):
        self.operation=operation
    @staticmethod
    def enabled(**kwargs):
        return True

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_data=X.X
        y_data=X.y
        
        print()
        print(f'{color.bdunl}XGBoost initiated{color.end}')
        #print(len(X_data))
        #print(len(y_data))
        
        # Get predictions using training and validation data
        xgp = process_model(X_data, y_data)
        xgp.osample()
        xgp.split_data(0.7)
        xgp_results = xgp.prep_run_model("Train/Valid Data Metrics",
                                        cmDisplay=True, PipeLine_flag = True)   
        
        #Test with unseen data
        print()
        print(f'{color.bdunl}Test With Unseen Data X_test and y_test{color.end}')
        
        xgp_model = xgp_results['xg_model']
        x_test = xgp_results['X_test']
        y_test = xgp_results['y_test']
        
        # Get predictions
        predictions = xgp_model.predict(x_test)
        cmv = model_eval(y_test, predictions)

        X.X = X_data
            
        '''
        A dictionary is returned, and its values can be used outside the pipeline if needed
        
        {'xg_model':xg_model,'predictions':predictions,
                    'X_train':X_train, 'y_train':y_train,
                    'X_valid':X_valid, 'y_valid':y_valid,
                    'X_test':X_test, 'y_test':y_test, 'cmv':cmv}
        '''
        return xgp_results

<a id="load_pl_df"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Load Dataset for PipeLine</b></div>

In [ ]:
X = pd.read_csv(f'{filepath}SBAnational.csv',
                 converters = {'DisbursementGross':fixvals,'SBA_Appv':fixvals,
                              'GrAppv':fixvals, 'ChgOffPrinGr':fixvals},
                              parse_dates=['DisbursementDate'], low_memory=False)
print("Shape of original SBA dataset : ", X.shape)
print()
print(X[['DisbursementGross','SBA_Appv','GrAppv','ChgOffPrinGr','DisbursementDate']].head(2))

# Filter data to before 2011
X = X[X['DisbursementDate'] <= '2010-12-31']

print()
print(f"Size of data after 2010-12-31 : \
    {len(X[X['DisbursementDate'] > '2010-12-31'])}")
print()
print(f"Size of data before 2011 : \
    {len(X[X['DisbursementDate'] < '2011-01-01'])}")

'''
X still contains the target 'MIS_Status', as we have to drop rows 
with NaNs in the pipeline. "MIS_Status" will be separated from X later in the pipeline

Select target - y is initialized as it goes into the pipeline, but will be updated in the pipeline 
after preprocessing.  Others preprocess y outside the pipeline; here, y will be preprocessed in
the pipeline.
'''
y = X['MIS_Status']

<a id="pl_run"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Run the pipeline</b></div>

In [ ]:
%%time

def RunPipeLine():
    rt1=dt.datetime.now()
    #Assign X and y to the object
    My_Object=PL_Object(X,y)

    #Build a simple pipeline
    My_Pipeline=Pipeline([('X Prep',PreProcessor('X')),
                          ('X EnCat',EncodeCategorical('X')),
                          ('y EnCat',EncodeCategorical('y')),
                          ('DropCols',DropColumns()),
                          ('XGBoost',XGBoost())
                         ])

    My_Object=My_Pipeline.transform(My_Object)

    print()
    print(f'{color.bdred}These results were obtained using Optuna tuning{color.end}')
    
    print()
    print(f'{color.bold}Pipeline Process Completed.{color.end}')

    rt2=dt.datetime.now()
    runtime(rt1,rt2)
    print()
    
    del My_Pipeline
    gc.collect()
    
    return My_Object        # for further usage below
    
MyObject = RunPipeLine()

<div class="alert alert-block alert-info" style="color:DarkSlateBlue">
    <b>Just for informative reasons</b>, below shows how we can use data (dictionary) passed back by the pipeline to My_Object
    </div>      

In [ ]:
def obj_sample_usage():
    print(MyObject.keys())
    pl_model = MyObject['xg_model']
    x=plot_features(pl_model, (10,14))
    print()
    MyObject['X_train'].info()
    
obj_sample_usage()

In [ ]:
# clear some variables from memory
del X, y, MyObject
gc.collect()

In [12]:
if alert_flag == 1:
    if kaggle_flag == 0:   # not Kaggle
        engine.say("SBA Machine Learning PipeLine completed.")
        engine.runAndWait()
    else:
        display(Audio(url=audio_path, autoplay=True))

<a id="part2"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 style='color:GhostWhite;'>Part 2 : Data Exploration and Preparation, Modeling, Metrics</h1></div>

<a id="de_load_df"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h3 style='color:GhostWhite;'>1. Load Dataset</h3></div>

In [ ]:
sba = pd.read_csv(f'{filepath}SBAnational.csv', low_memory=False)

print(sba.info(memory_usage = 'deep'))

<a id="dep"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2. Data Exploration / Preparation</h2><br>
    </div>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Reload dataset with some conversion</b><br>
    After review, decided to reload dataset with conversion of some features that may be needed for calculation.  It could be done after loading, but this is for instructive purposes on how it's done.
    </div>

In [ ]:
sba = pd.read_csv(filepath + 'SBAnational.csv',
                 converters = {'DisbursementGross':fixvals,'SBA_Appv':fixvals,
                              'GrAppv':fixvals, 'ChgOffPrinGr':fixvals},
                              parse_dates=['DisbursementDate'],
                              low_memory=False)

# Convert dtype of some columns that will be used in calculation or string extraction
sba = sba.astype({'DisbursementGross':np.float64,'SBA_Appv':np.float64,
                              'GrAppv':np.float64, 'ChgOffPrinGr':np.float64, 'NAICS':np.str_})

print(sba.info(memory_usage = 'deep'))
sba.head(1)

<a id="conv_dtype"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.1 EDA Tools</h2>
    </div>

## SweetViz

In [ ]:
htmlpath = f'{savepath}SBA_sweetviz_report_before.html'
GetSweetVizReport(sba,htmlpath,kaggle_flag)

print()
(kaggle_flag == 1) and create_download_link('Open SweetViz Report in browser ---> ',\
                                            f'{htmlpath}')

## Pandas Profiler
- this seems to be buggy as at Apr 2022

In [ ]:
'''
For a better experience, the report is created as an html file that can be opened in a browser,
and downloaded from there  (Save As ..., html)
'''
def GetPandasProfiling():
    print(f'{color.bdblue}Please wait ... Profiling Report will take some time.{color.end}')

    # uncomment if one wants to see the report in a cell below
    # df.profile_report(title='SBA Pandas Profiling Report')

    try:
        df = sba.copy()
        profile = df.profile_report(title='SBA Pandas Profiling Report', progress_bar=False,
                                    correlations={
                                        "pearson": {"calculate": True},
                                        "spearman": {"calculate": True},
                                        "kendall": {"calculate": False},
                                        "phi_k": {"calculate": True}
                                        })
        profile.to_file(output_file = f'{savepath}SBA_Profiling_Report.html')
        print(f'{color.bdblue}Profiling Report completed.{color.end}')
        print()
        (kaggle_flag == 0) and print(f'SBA Profiling Report has been downloaded to path {savepath}')
        
    except Exception as e:
        print(f'Error: {e}')

'''
GetPandasProfiling()

clear_output(wait=True)
gc.collect()
print()
(kaggle_flag == 1) and create_download_link('Open SBA Profiling Report in browser ---> ', \
                           f'{savepath}SBA_Profiling_Report.html')
'''

## DataPrep
- this also seems buggy

In [ ]:
from dataprep.datasets import load_dataset
from dataprep.eda import create_report

def GetDataPrepReport():
    print('Please wait, generating DataPrep report')
    try:
        df = sba.copy()
        report = create_report(df, title='SBA DataPrep Report', progress = False);
        report.save(f'{savepath}sba_dataprep_report')
    
        (kaggle_flag == 0) and report.show_browser()
        
    except Exception as e:
        print(f'Error: {e}')
        
'''
GetDataPrepReport()
clear_output(wait=True)
gc.collect()

# open html in browser, and from there, one can download using Save As, html   
(kaggle_flag == 1) and create_download_link('SBA DataPrep Report in browser ---> ',\
                                             f'{savepath}sba_dataprep_report.html');
'''

<a id="drop_rows_cols"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.2 Drop rows or columns if needed</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check for na's in all columns, as well as invalid categories</b></div>

In [ ]:
check_cols_with_nulls(sba)

In [ ]:
print(f'{color.bdunl}Features with NA values{color.end}')
sba.isna().sum()

**The number of Na's in rows for the following features, with respect to the size of the database, are not many and can be dropped.**

In [ ]:
sba.dropna(subset=['DisbursementDate', 'NewExist', 'City', 'State',
                        'LowDoc', 'Name', 'NAICS', 'CreateJob', 'RetainedJob', 'FranchiseCode',
                        'UrbanRural', 'NoEmp', 'Term', 'MIS_Status'], how='any', inplace=True)      

In [ ]:
sba.isna().sum()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>RevLineCr</b></div>

In [ ]:
len(sba[(sba['RevLineCr'] != 'Y') & (sba['RevLineCr'] != 'N')])
# too many unknowns, we will drop 'RevlineCr' later

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>LowDoc</b></div>

In [ ]:
len(sba[(sba['LowDoc'] != 'Y') & (sba['LowDoc'] != 'N')])

In [ ]:
sns.countplot(x='LowDoc',data=sba)

* **LowDoc seems to have a bearing**

In [ ]:
# we can drop rows that are not 'Y' or 'N'
sba = sba[(sba['LowDoc'] == 'Y') | (sba['LowDoc'] == 'N')]
len(sba[(sba['LowDoc'] != 'Y') & (sba['LowDoc'] != 'N')])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>NewExist</b>

In [ ]:
len(sba[(sba['NewExist'] != 1) & (sba['NewExist'] != 2)])

In [ ]:
sns.countplot(x='NewExist',data=sba)

In [ ]:
# records that are not 1 or 2, we can drop these rows as NewExist seems to have a bearing
sba = sba[(sba['NewExist'] == 1) | (sba['NewExist'] == 2)]
len(sba[(sba['NewExist'] != 1) & (sba['NewExist'] != 2)])

In [ ]:
sba = sba.astype({'NewExist':np.int8})

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>FranchiseCode</b></div>

In [ ]:
sba['FranchiseCode'].unique()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>UrbanRural</b></div>

In [ ]:
sba['UrbanRural'].unique()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Term</b></div>

In [ ]:
print(len(sba[sba['Term'].isna()]))
print(len(sba[sba['Term']==0]))
print(len(sba[sba['Term']<0]))

In [ ]:
sba.head(2)

In [ ]:
# Trim leading and trailing spaces
sba['City'] = sba['City'].str.strip()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check for na's in all columns</b></div>

In [ ]:
check_cols_with_nulls(sba)

# We can ignore these, features to be dropped later

In [ ]:
len(sba)

In [ ]:
# Save 2
def Save2():
    # for feather format, reset_index(drop=True) to prevent "Unnamed column" being created
    sdf = sba.copy().reset_index(drop=True)
    sdf.to_feather(f'{savepath}sba_save2.csv.feather')

    # index=False to prevent "Unnamed Column" being created
    #sba.to_csv(f'{savepath}sba_save2.csv', index=False)
    
    print(f'Saved to {savepath}sba_save2.csv.feather')

Save2()

# Short circuiting
(kaggle_flag == 1) and FileLink(r'sba_save2.csv.feather')  # Kaggle only

<a id="drop_duplicates"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.3 Drop Duplicate Rows</h2>
    </div>

In [ ]:
def DropDuplicates():
    dupl_series = sba.duplicated()
    num_of_dupl = len(sba[dupl_series == True])
    if num_of_dupl > 0:
        print(f'Number of Duplicates : {color.bold}{num_of_dupl}{color.end}')
        print()
        print(sba[dupl_series].head(5))
        sba.drop_duplicates(inplace=True)
        print()
        print(f'{color.bold}{num_of_dupl}{color.end} duplicate rows were dropped.')
    else:
        print(f'Duplicate rows found: {color.bold}None{color.end}')

DropDuplicates()

<a id="create_new_features"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.4 Create New Features</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Industry</b> - The industry sector is the 1st 2 digits of NAICS
    </div>

In [ ]:
sba['Industry'] = sba['NAICS'].str[0:2]
sba = sba.astype({'Industry':np.int32})

In [ ]:
sba['Industry'].head(2)

In [ ]:
sba['Industry'].unique()
# There is an invalid industry shown which is '0', caused by blank NAICS

In [ ]:
len(sba[sba['Industry'] == 0])
# This is a bummer, as industry sector has a big effect on a business, speaking as a business 
# domain expert.  Do we drop those with NAICS = 0 ?

In [ ]:
# At this stage, we leave it as is and treat it as unknown industry
sba.head(2)

In [ ]:
# Check if we can impute from the name.  For example, a bar (or similar) business
sba[(sba['Name'].str.contains('bar',case=False)) & (sba['Industry'] == 0)]\
    [['Name','Industry']].head(10)

**It's not feasible to impute missing Industry codes efficiently, so we abandon the idea.**

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Recession</b><br>
We want to account for variation due to the Great Recession (December 2007 to June 2009). Should we separate the datasets into different time periods ? Before, During, and After ?  Let's check how large the sets are later.  In the meantime, we create a new feature, Recession, with 1 for 'Y' and 0 for 'N' depending on the DisbursementDate. 
<br><br>
</div>

In [ ]:
# Convert "DisbursementDate" to datetime

# sba['DisbursementDate'] = pd.to_datetime(sba['DisbursementDate'], format='%d-%b-%y')

# sba.head(2)

In [ ]:
# Create new column based on condition
sba['Recession'] = np.where((sba['DisbursementDate'] >= '2007-09-01')\
                     & (sba['DisbursementDate'] <= '2009-06-30'), 1, 0)

In [ ]:
print(f'Total - {len(sba)}')
y = len(sba[sba['Recession'] == 1])
n = len(sba[sba['Recession'] == 0])
print(f'Recession - {y}')
print(f'Not Recession - {n}')

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Real Estate</b><br>
Loans backed by real estate will have terms 20 years or greater (≥240 months) and are the only loans granted for such a long term, whereas loans not backed by real estate will have terms less than 20 years ( < 240 months).<br><br>
1 - Backed By Real Estate<br>
0 - Not Backed By Real Estate<br><br>

In [ ]:
# Create new column based on condition
sba['RealEstate'] = np.where(sba['Term'] >= 240, 1, 0)

In [ ]:
print(f'Total - {len(sba)}')
y = len(sba[sba['RealEstate'] == 1])
n = len(sba[sba['RealEstate'] == 0])
print(f'Yes - {y}')
print(f'No - {n}')
print(f'Yes and No - {y+n}')

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>SBA_Portion</b><br>
The portion which is the percentage of the loan that is guaranteed by SBA. This is derived by calculating the ratio of the amount of the loan SBA guarantees and the gross amount approved by the bank (SBA_Appv/GrAppv) * 100.<br><br></div>

In [ ]:
sba['SBA_Portion']=(sba['SBA_Appv']/sba['GrAppv']) * 100
sba.head(2)

**CityState**

In [ ]:
sba["CityState"] = sba["City"] + "_" + sba["State"]
sba[["CityState", "City", "State"]].head()

In [ ]:
sba.head(2)

In [ ]:
# Save 3
def Save3():
    sdf = sba.copy().reset_index(drop=True)
    sdf.to_feather(f'{savepath}sba_save3.csv.feather')

    print(f'Saved to {savepath}sba_save3.csv.feather')
    
Save3()

(kaggle_flag == 1) and FileLink(r'sba_save3.csv.feather')  # Kaggle only

<a id="encode_cat"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.5 Encode Categorical Features</h2>
    </div>

In [ ]:
sba.select_dtypes(["object"]).nunique()

<div style="font-family: Trebuchet MS;background-color:Chocolate;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>MIS_Status</b><br>
    This will be the <b>target</b> variable</div>

In [ ]:
sns.set_style('whitegrid')
# Target variable is MIS Status, a categorical variable

print(sba['MIS_Status'].value_counts())
sns.countplot(x='MIS_Status',data=sba)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    This shows a skewed distribution, where this bias in the target can influence many machine learning algorithms, leading some to ignore the minority class entirely, in this case, CHGOFF.  Before oversampling the data, will try as is.<br><br></div>

In [ ]:
# Update column based on condition
sba['MIS_Status'] = np.where((sba['MIS_Status'] == 'P I F'), 1, 0)

In [ ]:
print(sba['MIS_Status'].dtype)
sba.head(2)[['City','MIS_Status']]

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>LowDoc</b><br>
'Y' = 1<br>
'N' = 0

In [ ]:
# Update column based on condition
sba['LowDoc'] = np.where((sba['LowDoc'] == 'Y'), 1, 0)

sba.head(2)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Others</b></div>

In [ ]:
# will not hash 'City' as it is already covered by 'CityState'

def HashCol():
    cols_to_drop = []
    hash_constant = 900000   # fixed value so we can programmatically reproduce the hash when needed
    len_data=len(sba)
    for col in sba[['State','CityState']]:
        if sba[col].dtype == 'object':
            print(f'Column {col} has {sba[col].nunique()} values among {len_data}')

        if sba[col].nunique() < 25:
            print(f'One-hot encoding of {col}')
            one_hot_cols = pd.get_dummies(sba[col])
            for ohc in one_hot_cols.columns:
                sba[col + '_' + ohc] = one_hot_cols[ohc]
        else:
            print(f'Hashing of {col}')
            sba[col + '_hash'] = sba[col].apply(lambda row: int(hashlib.sha1((col + "_" + \
                                    str(row)).encode('utf-8')).hexdigest(), 16) % hash_constant)

        cols_to_drop.append(col)
    print(cols_to_drop)

HashCol()

In [ ]:
sba.head(2)[['State','CityState','State_hash','CityState_hash']]

In [ ]:
sba.info()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>TimeFrame</b><br>
Create a dataset for later use where we restrict the time frame to loans by excluding those disbursed after 2010 due to the fact the term of a loan is frequently 5 or more years.
    <br><br>

In [ ]:
sba_bef_2011 = sba[sba['DisbursementDate'] <= '2010-12-31'].copy()
len(sba_bef_2011[sba_bef_2011['DisbursementDate'] > '2010-12-31'])
len(sba_bef_2011[sba_bef_2011['DisbursementDate'] <= '2011-01-01'])

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Drop columns that are no longer needed<b></div>

In [ ]:
# Save 4
def Save4():
    sdf = sba.copy().reset_index(drop=True)
    sdf.to_feather(f'{savepath}sba_save4.csv.feather')

    print(f'saved to {savepath}sba_save4.csv.feather')

Save4()

(kaggle_flag == 1) and FileLink(r'sba_save4.csv.feather')  # Kaggle only

In [ ]:
cols_to_drop = ['LoanNr_ChkDgt', 'Bank', 'BankState', 'ApprovalDate',
                        'ApprovalFY', 'ChgOffDate', 'BalanceGross', 'NAICS', 'ChgOffPrinGr',
                        'Name', 'RevLineCr', 'DisbursementDate', 'City', 'State', 'CityState',
                         'GrAppv','Zip']

sba_bef_2011.drop(columns=cols_to_drop, inplace=True)

sba.drop(columns=cols_to_drop, inplace=True)

sba_bef_2011 = reduce_mem_usage(sba_bef_2011)

print()
sba = reduce_mem_usage(sba)

print()
print('Unneeded Columns Dropped')
print(sba.info())

In [ ]:
def SaveBef2011():
    # Save sba_bef_2011
    ## save this dataset to working dir
    sdf = sba_bef_2011.copy().reset_index(drop=True)
    sdf.to_feather(f'{savepath}sba_bef_2011.csv.feather')

    print(f"saved to {savepath}sba_bef_2011.csv.feather")
    
SaveBef2011()

(kaggle_flag == 1) and FileLink(r'sba_bef_2011.csv.feather')  # Kaggle only

In [ ]:
del sba_bef_2011
gc.collect()
sleep(5)

In [ ]:
# Save 5
def Save5():
    sdf = sba.copy().reset_index(drop=True)
    sdf.to_feather(f'{savepath}sba_save5.csv.feather')

    print(f'saved to {savepath}sba_save5.csv.feather')

Save5()

(kaggle_flag == 1) and FileLink(r'sba_save5.csv.feather')  # Kaggle only

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check for Infinite Values<b></div>

In [ ]:
check_infinity_nan(sba,'sba')

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Check Correlations</b></div>

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))

g = sns.heatmap(
    sba.corr(),
    annot=True,
    ax=ax,
    cmap='OrRd',
    cbar=False,
    linewidth=1
)

g.set_xticklabels(g.get_xticklabels(), rotation=45, horizontalalignment='right')
g.set_yticklabels(g.get_yticklabels(), rotation=45, horizontalalignment='right')

<a id="eda_check"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.6 EDA Check</h2><br>
    Here we generate a SweetViz report for another EDA review
    </div>

In [ ]:
htmlpath = f'{savepath}SBA_sweetviz_report_after.html'
GetSweetVizReport(sba,htmlpath,kaggle_flag)

print()
(kaggle_flag == 1) and create_download_link('Open SweetViz Report in browser ---> ',\
                                            f'{htmlpath}')

In [ ]:
del sba
gc.collect()
sleep(5)

<a id="build_model"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3. Build Model Using XGBoost</h2>
    </div>

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Early Stopping Rounds<b></div>

"<b>Overfitting</b> is a problem with sophisticated non-linear learning algorithms like gradient boosting.  Early stopping is an approach to training complex machine learning models to avoid overfitting.
<br><br>
<b>XGBoost supports early stopping after a fixed number of iterations.</b>  In addition to specifying a metric and test dataset for evaluation in each epoch, one must specify a window of the number of epochs over which no improvement is observed. This is specified in the early_stopping_rounds parameter.
<br><br>
It is generally a good idea to select the early_stopping_rounds as a reasonable function of the total number of training epochs (10% in this case) or attempt to correspond to the period of inflection points as might be observed on plots of learning curves.
<br><br> - <a href = "https://machinelearningmastery.com/avoid-overfitting-by-early-stopping-with-xgboost-in-python/">Avoid Overfitting By Early Stopping With XGBoost In Python</a>

<a id="model1"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3.1 Model v1</h2>
    </div>

In [ ]:
%%time

def RunModelv1():
    # Select subset of predictors
    X = pd.read_feather(f'{savepath}sba_save5.csv.feather')
    y = X.pop('MIS_Status')

    model1 = process_model(X, y)   # Initiate class
    model1.split_data(0.7)         # Split data into train (70%), valid (15%), and test (15%)
    
    params = {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 6,
                               'tree_method':tree_method, 'early_stopping_rounds':100,
                               'eval_metric':['auc','error']}
    model1.prep_run_model( "Metrics : Full SBA Not Oversampled",hyperparams=params)
    print()
    
RunModelv1()

<div class="alert alert-block alert-danger">  
<b>Accuracy for model is good; but ...</b> Precision, Recall, and f1-score of minority class 0 (CHGOFF) is <b>much lower</b> than that of 1 (P I F). This is because MIS_Status is heavily skewed towards 1 (P I F).</div>

<div class="alert alert-block alert-info">  
In such a scenario, <b>Accuracy is not a good metric</b>, as it favors the majority.  <b>The f1-score is the more ideal metric</b>, which correctly shows a poorer score by the minority class.<br><br>
    To solve this, we try <b>Oversampling the data</b>, in the next section.</div>

<a id="oversample"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;"><h2 style='color:GhostWhite;'>3.2 OverSample</h2>
    </div>

<a id="model2"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;"><h2 style='color:GhostWhite;'>3.2.1 Model v2</h2>
    </div>

In [ ]:
%%time

def RunModelv2():
    # Select subset of predictors
    X = pd.read_feather(f'{savepath}sba_save5.csv.feather')

    # Select target
    y = X.pop('MIS_Status')

    model2 = process_model(X, y)
    model2.osample()             # oversampling method
    model2.split_data(0.7)
    model2_results = model2.prep_run_model("Metrics : Full SBA Oversampled")

    return model2_results['xg_model']

modelv2 = RunModelv2()

<div class="alert alert-block alert-info">
    After oversampling of the minority class, class 0 (CHGOFF) <b>now has a similar </b> precision, recall, and f1-score as class 1 (P I F).
<br><br>     
    The <b>accuracy score</b> is slightly lower than when not oversampled, but much better f-scores.  This should be a good metric now as the target classification is no longer imbalanced.  <b>The model is now more accurate in predicting the target as f-scores are better now.</b></div>

In [ ]:
# Plot feature importance

plot_features(modelv2, (10,12))

<div class="alert alert-block alert-info">
    <b>Observation</b><br>
    I was hoping to see <b>Industry</b> at a much higher position here, but apparently the incomplete data on industry had an effect.<br><br>
Furthermore, <b>Recession</b> has to be at a very high position, but is at the bottom instead.  This could be due to <b>Recession</b> data being highly skewed towards 1 (Not Recession).<br><br>
<b>Real Estate</b> should have good importance too, but it may be highly skewed as well.

In [ ]:
del modelv2
gc.collect()

<a id="model3"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>3.2.2 Model v3</h2>
    <b>Build a Model Dataset Excluding Year 2011 and Above</b>

We restrict the time frame to loans by excluding those disbursed after 2010 due to the fact the term of a loan is frequently 5 or more years.
       </div>

In [ ]:
%%time

def RunModelv3():
    X = pd.read_feather(f'{savepath}sba_bef_2011.csv.feather')
    y = X.pop('MIS_Status')
    
    memuse = formatFileSize(X.memory_usage().sum(),'B','MB',2)

    model3 = process_model(X, y)
    model3.osample()
    model3.split_data(0.7)
    
    model3.X, model3.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    model3_results = model3.prep_run_model("Metrics : SBA Before 2011 Oversampled")
    
    # save to files for reuse later
    model3_results['xg_model'].save_model(f'{savepath}modelv3.json')
    joblib.dump(model3_results, f"{savepath}model3_results.dict")   # save study
    
    return model3_results
  
model3_results = RunModelv3()

<div class="alert alert-block alert-info">
    <b>We get a similar score to Model 2.</b>  Will use this dataset as the last dataset, for now.</div>

In [ ]:
# Save 6 - final cleaned csv
def SaveFinalCsv():
    #sdf = pd.read_csv(f'{savepath}sba_bef_2011.csv')
    #sdf.to_csv(f'{savepath}sba_final.csv', index=False)
    
    src_file=f'{savepath}sba_bef_2011.csv.feather'
    dst_file=f'{savepath}sba_final.csv.feather'
    shutil.copy2(src_file, dst_file)
    
    #save to csv
    df = pd.read_feather(dst_file)
    df.to_csv(f'{savepath}sba_final.csv', index=False)

    print(f'saved to {savepath}sba_final.csv.feather and {savepath}sba_final.csv')

SaveFinalCsv()
(kaggle_flag == 1) and FileLink(r'sba_final.csv.feather')  # Kaggle only

**Next two cells are just comparing results of two different ways of loading a saved dictionary file.  Result should be the same.** 

In [ ]:
# Plot feature importance
modelv3 = XGBClassifier()
modelv3.load_model(f'{savepath}modelv3.json')
plot_features(modelv3, (10,14))

In [ ]:
model3_results = joblib.load(f"{savepath}model3_results.dict")
plot_features(model3_results['xg_model'], (10,14))

<a id="test_model"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>4. Test Model</h2>
    </div>
    

<a id="test_test_dataset"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>4.1 Test Model with Test Dataset</h2>
    Test Dataset was previously unseen by the model.
    </div>

In [ ]:
def Modelv3WithTestData():
    X_test = model3_results['X_test']
    y_test = model3_results['y_test']
    
    # Get predictions
    predictions = modelv3.predict(X_test)
    model_eval(y_test, predictions);
  
Modelv3WithTestData()

<a id="test_user_input"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>4.2 Test Model with User Input</h2>
    </div>

<div class="alert alert-block alert-info">So let's assume the following are <b>the entries of a user</b>, through a user interface, looking for a prediction from our model.</div>

In [ ]:
def UserInputTest():
    # 16 entries
    user_input =   {'Term':50, 
                    'NoEmp':0,
                    'NewExist':1,
                    'CreateJob':0 ,          
                    'RetainedJob':0,         
                    'FranchiseCode':1,       
                    'UrbanRural':0,           
                    'LowDoc':0,               
                    'DisbursementGross':50000,                 
                    'SBA_Appv':25000,          
                    'Industry':71, 
                    'Recession':0,
                    'RealEstate':0,           
                    'SBA_Portion':50,
                    'City':'EVANSVILLE',
                    'State':'IN'
                   }

    city = user_input['City']
    state = user_input['State']
    city_state = f'{city}_{state}'

    state_hash = int(hashlib.sha1(('State' + "_" + \
                              str(state)).encode('utf-8')).hexdigest(), 16) % 900000
    city_state_hash = int(hashlib.sha1(('CityState' + "_" + \
                              str(city_state)).encode('utf-8')).hexdigest(), 16) % 900000

    print(f'State_hash = {state_hash}')
    print(f'CityState_hash = {city_state_hash}')

    user_input.pop('City')
    user_input.pop('State')
    user_input['State_hash'] = state_hash
    user_input['CityState_hash'] = city_state_hash

    user_input_list = list(user_input.values())
    
    return {'user_input':user_input, 'user_input_list':user_input_list}

user_input_param = UserInputTest()

print()
print(f"{color.bold}User Entry:{color.end}")
user_input_param['user_input']

In [ ]:
# User Input test 1
def UserInputTest1():
    features = np.array([user_input_param['user_input_list']])   

    # using inputs to predict the output
    pred = modelv3.predict(features)
    if pred[0] == 1:
        print(f'{color.bdblue}Prediction: Approve The Loan{color.end}')
    else:
        print(f'{color.bdred}Prediction: Do Not Approve The Loan{color.end}')
        
UserInputTest1()

In [ ]:
# User Input test 2
def UserInputTest2():
    '''
    # if one wants to edit the list from the previous cell
    user_input2_list = user_input_list[:]   # make a copy
    user_input2_list[0] = 500          # change term 
    '''

    user_input2 = copy.deepcopy(user_input_param['user_input'])
    user_input2['Term'] = 500     # change term
    user_input2_list = list(user_input2.values())

    features = np.array([user_input2_list]) 

    # using inputs to predict the output
    pred = modelv3.predict(features)
    if pred[0] == 1:
        print(f'{color.bdblue}Prediction: Approve The Loan{color.end}')
    else:
        print(f'{color.bdred}Prediction: Do Not Approve The Loan{color.end}')
        
UserInputTest2()

<div style="font-family: Trebuchet MS;background-color:HoneyDew;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;border: 5px solid CadetBlue;"><b>Predictions:</b><br>
    
- 1 -> can approve<br>
- 0 -> do not approve<br>

Of course, in real life, will need to check further using other data (e.g. financial statements, kind of real estate, etc.) or other data's models if available.

In [ ]:
del user_input_param
gc.collect()

<a id="mutual_info"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>5. Mutual Information Scores</h2>
 "A general-purpose metric, normally used before selecting and building a model, but used here in the end, for comparison.  Mutual information is a lot like correlation in that it measures a relationship between two quantities. The advantage of mutual information is that it can detect any kind of relationship, while correlation only detects linear relationships."
    </div>

In [ ]:
%%time

def GetMIScores():
    X = pd.read_feather(f'{savepath}sba_final.csv.feather')
    y = X.pop('MIS_Status')

    model_mi = process_model(X, y)
    model_mi.osample()
    
    del X,y
    gc.collect()
    sleep(3)
    
    mi_scores = make_mi_scores(model_mi.X, model_mi.y)

    print()
    
    return mi_scores

mi_scores = GetMIScores()

In [ ]:
plt.figure(dpi=1200, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
# Plot feature importance
plot_features(modelv3, (10,14))

In [ ]:
del mi_scores
gc.collect()

In [ ]:
if alert_flag == 1:
    if kaggle_flag == 0:   # not Kaggle
        engine.say("SBA Mutual Information completed.")
        engine.runAndWait()
    else:
        display(Audio(url=audio_path, autoplay=True))

<div class="alert alert-block alert-info">
The importance ranked by <b>Mutual Information</b> and <b>XGBoost Feature Importance</b> metrics are different.  Which ranking do you think is more reasonable ?</div>

<a id="trim_dataset"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>6. Trim Dataset</h2><br>
After the preprocessing and encoding steps, not all of the features may be useful in forecasting the loan default. Alternatively we can select the <b>top 5 or top 8 features</b>, based on the feature importance plot above, which had a major contribution in forecasting loan defaults.<br><br>

If the model performance is similar in both the cases, that is – by using all the features and by using 5-8 features, then we should use only the top 8 features, in order to keep the model simpler and more efficient.

The idea is to have a less complex model without compromising on the overall model performance.
</div>

In [ ]:
X = pd.read_feather(f'{savepath}sba_final.csv.feather')
y = X.pop('MIS_Status')

#Let's retain the top 8 from Mutual Information metric 
mi_features = ['Term', 'DisbursementGross', 'SBA_Appv', 'SBA_Portion',
                'CityState_hash', 'FranchiseCode', 'RealEstate', 'UrbanRural']

Xmi = X[mi_features]

#Let's retain the top 8 from Feature Importance metric 
fi_features = ['Term', 'SBA_Appv', 'DisbursementGross', 'CityState_hash', 'State_hash',
                'SBA_Portion', 'Industry', 'NoEmp']

Xfi = X[fi_features]

del X
gc.collect()

In [ ]:
%%time

def ModelMI():
    model_mi = process_model(Xmi, y)
    model_mi.osample()
    model_mi.split_data(0.7)
  
    model_mi_results = model_mi.prep_run_model("Mutual Information Metrics")
    
    print()
    return model_mi_results

model_mi_results = ModelMI()

In [ ]:
# Plot mutual information
my_model_mi = model_mi_results['xg_model']
plot_features(my_model_mi, (10,5))

In [ ]:
# Test with Unseen test data
def MI_Model_On_Test_Data():
    X_test = model_mi_results['X_test']
    X_test_mi = X_test[mi_features]

    y_test = model_mi_results['y_test']

    predictions_mi = my_model_mi.predict(X_test_mi)
    model_eval(y_test, predictions_mi)
    print()
    
MI_Model_On_Test_Data()

In [ ]:
del Xmi, mi_features, my_model_mi, model_mi_results
gc.collect()

In [ ]:
%%time

def ModelFI():
    model_fi = process_model(Xfi, y)
    model_fi.osample()
    model_fi.split_data(0.7)
    
    model_fi_results = model_fi.prep_run_model("Feature Importance Metrics")

    print()
    return model_fi_results
    
model_fi_results = ModelFI()

In [ ]:
# Plot feature importance
my_model_fi = model_fi_results['xg_model']
plot_features(my_model_fi, (10,5))

In [ ]:
# Test with Unseen test data
def FI_Model_On_Test_Data():
    X_test = model_fi_results['X_test']
    X_test_fi = X_test[fi_features]

    y_test = model_fi_results['y_test']

    predictions_fi = my_model_fi.predict(X_test_fi)
    model_eval(y_test, predictions_fi)
    print()
    
FI_Model_On_Test_Data()

In [ ]:
del Xfi, fi_features, my_model_fi, model_fi_results
del y, model3_results, modelv3
gc.collect()

<a id="results1"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>7. Full or Trimmed Dataset</h2>
</div>

<div class="alert alert-block alert-info">
    <b>Do we select the full dataset, or the trimmed dataset ?</b><br><br>
    <b>Observation:</b><br>
    <ul>
        <li><b>Accuracy</b> - Approx 2 points less accuracy of trimmed versus the full features dataset.<br><br>
        <li><b>f1-score</b> - Also approx 2 points less f1-score between full features dataset and Manual Information trimmed dataset.  Approx 1 point difference between full features dataset and Feature Importance trimmed dataset.<br><br>
    </ul>
    We can <b>stick with the full features</b> for now; but the trimmed features are also good, with the <b>Manual Information trimmed dataset</b> very slightly favored.